In [9]:
INPUT_DIR = "../img/"

# Watersheds Segmentation <a href="https://mybinder.org/v2/gh/InsightSoftwareConsortium/SimpleITK-Notebooks/master?filepath=Python%2F32_Watersheds_Segmentation.ipynb"><img style="float: right;" src="https://mybinder.org/badge_logo.svg"></a>

In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
import SimpleITK as sitk
from myshow import myshow, myshow3d

# Download data to work on
# %run update_path_to_download_script
# from downloaddata import fetch_data as fdata

In [11]:
img = sitk.ReadImage(INPUT_DIR + "DICOM3.mha")
myshow(img)

interactive(children=(IntSlider(value=205, description='z', max=411), Output()), _dom_classes=('widget-interac…

## Gradient Watersheds Segmentation

In [12]:
sigma = img.GetSpacing()[0]
level = 4

In [13]:
feature_img = sitk.GradientMagnitude(img)
myshow(feature_img, "Edge Features")

interactive(children=(IntSlider(value=205, description='z', max=411), Output()), _dom_classes=('widget-interac…

In [14]:
ws_img = sitk.MorphologicalWatershed(
    feature_img, level=0, markWatershedLine=True, fullyConnected=False
)
myshow(sitk.LabelToRGB(ws_img), "Watershed Over Segmentation")

interactive(children=(IntSlider(value=205, description='z', max=411), Output()), _dom_classes=('widget-interac…

In [15]:
from ipywidgets import interact, interactive, FloatSlider


def callback(feature_img, *args, **kwargs):
    ws_img = sitk.MorphologicalWatershed(feature_img, *args, **kwargs)

    myshow(sitk.LabelToRGB(ws_img), "Watershed Segmentation")


interact(
    lambda **kwargs: callback(feature_img, **kwargs),
    markWatershedLine=True,
    fullyConnected=False,
    level=FloatSlider(min=0, max=255, step=0.1, value=4.0),
)

interactive(children=(Checkbox(value=True, description='markWatershedLine'), Checkbox(value=False, description…

<function __main__.<lambda>(**kwargs)>

## Segmentation From Markers

In [16]:
min_img = sitk.RegionalMinima(
    feature_img,
    backgroundValue=0,
    foregroundValue=1.0,
    fullyConnected=False,
    flatIsMinima=True,
)
marker_img = sitk.ConnectedComponent(min_img)
myshow(sitk.LabelToRGB(marker_img), "Too many local minima markers")

interactive(children=(IntSlider(value=205, description='z', max=411), Output()), _dom_classes=('widget-interac…

In [17]:
ws = sitk.MorphologicalWatershedFromMarkers(
    feature_img, marker_img, markWatershedLine=True, fullyConnected=False
)
myshow(sitk.LabelToRGB(ws), "Watershed Oversegmentation from markers")

interactive(children=(IntSlider(value=205, description='z', max=411), Output()), _dom_classes=('widget-interac…

In [18]:
pt = [60, 60]
idx = img.TransformPhysicalPointToIndex(pt)
marker_img *= 0
marker_img[0, 0] = 1
marker_img[idx] = 2

ws = sitk.MorphologicalWatershedFromMarkers(
    feature_img, marker_img, markWatershedLine=True, fullyConnected=False
)
myshow(
    sitk.LabelOverlay(img, ws, opacity=0.2),
    "Watershed Oversegmentation from manual markers",
)

RuntimeError: Exception thrown in SimpleITK Image_TransformPhysicalPointToIndex: ../../Code/Common/src/sitkPimpleImageBase.hxx:201:
sitk::ERROR: vector dimension mismatch

## Binary Watersheds for Object Separation 

In [19]:
rgb_img = sitk.ReadImage(INPUT_DIR + "DICOM3.mha")
myshow(rgb_img, INPUT_DIR + "DICOM3.mha")
img = sitk.VectorIndexSelectionCast(rgb_img, 1)
myshow(img, INPUT_DIR + "covid19.mha")

interactive(children=(IntSlider(value=205, description='z', max=411), Output()), _dom_classes=('widget-interac…

RuntimeError: Exception thrown in SimpleITK VectorIndexSelectionCast: ../../Code/Common/include/sitkDualMemberFunctionFactory.hxx:193:
sitk::ERROR: Pixel type: 32-bit signed integer is not supported in 3D byN3itk6simple35VectorIndexSelectionCastImageFilterE

In [20]:
feature_img = sitk.GradientMagnitudeRecursiveGaussian(img, sigma=1.5)
myshow(feature_img)

interactive(children=(IntSlider(value=205, description='z', max=411), Output()), _dom_classes=('widget-interac…

In [21]:
ws_img = sitk.MorphologicalWatershed(
    feature_img, level=4, markWatershedLine=False, fullyConnected=False
)
myshow(sitk.LabelToRGB(ws_img), "Watershed Over Segmentation")
seg = sitk.ConnectedComponent(ws_img != ws_img[0, 0])
myshow(sitk.LabelOverlay(img, seg), "Foreground Components")

interactive(children=(IntSlider(value=205, description='z', max=411), Output()), _dom_classes=('widget-interac…

IndexError: invalid slice extraction to 1 dimension

In [22]:
filled = sitk.BinaryFillhole(seg != 0)
d = sitk.SignedMaurerDistanceMap(
    filled, insideIsPositive=False, squaredDistance=False, useImageSpacing=False
)
myshow(d, "Inside Distance Map")

NameError: name 'seg' is not defined

In [23]:
ws = sitk.MorphologicalWatershed(d, markWatershedLine=False, level=1)
myshow(sitk.LabelOverlay(img, ws))

NameError: name 'd' is not defined

In [ ]:
ws = sitk.Mask(ws, sitk.Cast(seg, ws.GetPixelID()))
myshow(sitk.LabelOverlay(img, ws), "Split Objects")

## Multi-label Morphology

In [ ]:
seg = ws

In [ ]:
radius = 10
bd_img = sitk.BinaryDilate(seg != 0, [radius] * seg.GetDimension())
myshow(bd_img, "Binary Dilate")

In [ ]:
dist_img = sitk.SignedMaurerDistanceMap(
    seg != 0, insideIsPositive=False, squaredDistance=False, useImageSpacing=False
)
wsd_img = sitk.MorphologicalWatershedFromMarkers(dist_img, seg, markWatershedLine=False)
myshow(sitk.LabelOverlay(img, wsd_img))

In [ ]:
md_img = sitk.Mask(wsd_img, bd_img)
myshow(sitk.LabelToRGB(md_img), "Multi-label Dilate")

In [ ]:
e_img = sitk.BinaryErode(md_img != 0, [radius] * md_img.GetDimension())
mo_img = sitk.Mask(md_img, e_img)
myshow(sitk.LabelOverlay(img, mo_img), "Multi-label Closing")